# Lab 3: Securely connect tools to your Agent with AgentCore Gateway 

## Overview

In this Lab, you will learn how to integrate tools with the Customer Service Assistant using Amazon Bedrock Gateway and the [Model Context Protocol (MCP)](https://modelcontextprotocol.io/docs/getting-started/intro).

[Amazon Bedrock Agent Core Gateway](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway.html) converts APIs, Lambda functions, and existing services into MCP-compatible tools with just a few lines of code.

**Workshop Journey:**
- **Lab 1 (Done):** Create Agent Prototype
- **Lab 2 (Done):** Enhance with Memory
- **Lab 3 (Current):** Scale with Gateway & Identity
- **Lab 4:** Deploy to Production
- **Lab 5:** Build User Interface

### Why AgentCore Gateway Matters

**Current State (Lab 1-2):** Each agent has its own copy of tools, leading to code duplication, inconsistent behavior, no centralized security, and scaling difficulties.

**After this lab:** Centralized, reusable tools that serve Customer Service, Sales, Inventory, and Returns Processing agents.

### Secure Authentication with AgentCore Identity

[AgentCore Identity](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/identity.html) provides agent identity and access management across AWS services and third-party applications, supporting standard identity providers (Okta, Entra, Amazon Cognito).

**Inbound Authentication:** Gateway validates OAuth tokens to allow/deny tool access.

**Outbound Authentication:** Gateway uses API Key, IAM, or OAuth to access external resources.


## Architecture for Lab 3

<div style="text-align:left">
    <img src="images/Intelligent Customer Service Assistant Lab3 - Gateway.png" width="100%"/>
</div>

*Order tracking tool is now centralized in AgentCore Gateway with secure identity-based access control. Multiple agents and use cases can share the same tool securely. We will also reuse the `get_customer_profile()` tool built for other applications and add the `track_order()` tool for use within other applications. 

### Key Features
- **Seamlessly integrate AWS Lambda functions:** This example shows how to integrate your Agent with existing AWS Lambda functions to track orders and get customer profiles using Amazon Bedrock AgentCore Gateway.
- **Secure your Gateway endpoint with Inbound Auth**: Only an Agent providing a valid JWT token can connect to the endpoint to use the tools
- **Configure the Agent to use the MCP endpoint**: The Agent gets a valid JWT token and uses it to connect to the MCP endpoint provided by AgentCore Gateway.

## Step 1: Install and import required libraries

In [ ]:
# Install required packages
%pip install strands-agents "boto3>=1.39.15" strands-agents-tools bedrock_agentcore ddgs -q --no-deps
print("[WHITE HEAVY CHECK MARK] Packages installed successfully!")

In [ ]:
# Import libraries
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
import os
import sys
import boto3
import json
from bedrock_agentcore.identity.auth import requires_access_token
from mcp.client.streamable_http import streamablehttp_client
import requests

from scripts.utils import get_ssm_parameter, put_ssm_parameter, load_api_spec, get_cognito_client_secret

sts_client = boto3.client('sts')

# Get AWS account details
REGION = boto3.session.Session().region_name

gateway_client = boto3.client(
    "bedrock-agentcore-control",
    region_name=REGION,
)

print("[WHITE HEAVY CHECK MARK] Libraries imported successfully!")

## Step 2: Give our agent a tool to access existing customer data and order tracking

AgentCore Gateway simplifies agent tool integration in three key ways:

Universal MCP Support: Instantly make your tools compatible with any agent framework by exposing them through AgentCore Gateway's MCP standard.

Simple REST Integration: Transform existing REST services into agent tools by just adding them as AgentCore Gateway targets.

Lambda Flexibility: Expose Lambda functions as MCP endpoints that can call any API - demonstrated here with a function that tracks orders.

AgentCore Gateway populates the Lambda context with the name of the tool to invoke, while the parameters passed to the tool are provided in the Lambda event:

Note: This Lambda function is already deployed in the workshop environment.

```
extended_tool_name = context.client_context.custom["bedrockAgentCoreToolName"]
resource = extended_tool_name.split("___")[1]
```

[Lambda function](./prerequisite/lambda/python/lambda_function.py) (Already deployed in the workshop environment)

```
def lambda_handler(event, context):
    if resource == "track_order":
        order_id = get_named_parameter(event=event, name="order_id")
        tracking_id = get_named_parameter(event=event, name="tracking_id")
        customer_id = get_named_parameter(event=event, name="customer_id")
        natural_query = get_named_parameter(event=event, name="natural_query")

        order_status = track_order(order_id, tracking_id, customer_id, natural_query)
        return {"statusCode": 200, "body": order_status}
```

print("[WHITE HEAVY CHECK MARK] Lambda function defined.")

## Step 3 Convert your order tracking tool to MCP
Now that we are developing an MCP server using AgentCore Gateway, we can MCP-ify any tools which we think we'll use for multiple Agents. One of these tools is order tracking which can be used across different customer service scenarios. As a result, we converted the order tracking tool into a Lambda tool within our AgentCore Gateway:

Note: This Lambda function is already deployed in the workshop environment.

[Order tracking Lambda](./prerequisite/lambda/python/track_order.py) (Already deployed in the workshop environment)
```
def track_order(order_id: Optional[str] = None, tracking_id: Optional[str] = None, 
                customer_id: Optional[str] = None, natural_query: Optional[str] = None) -> str:
    """Track order status using various identifiers or natural language query
    
    Args:
        order_id: Specific order ID to look up
        tracking_id: Tracking ID to look up
        customer_id: Customer ID to filter orders
        natural_query: Natural language query about the order
        
    Returns:
        Formatted string with order status information
    """
    # Implementation handles DynamoDB queries and natural language processing
    # Supports queries like "my iPhone order from last Friday"
    # Returns formatted order status with tracking information


print("[WHITE HEAVY CHECK MARK] Order tracking tool ready")
```

## Step 4 Create your function definition metadata
Lastly, we need to write tool schema which describes the tools implemented by your Lambda function.

This file has been already defined in [prerequisite/lambda/api_spec.json](./prerequisite/lambda/api_spec.json)

```
[
    {
        "name": "get_customer_profile",
        "description": "Retrieve customer profile using customer ID, email, or phone number",
        "inputSchema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "string"
                },
                "email": {
                    "type": "string"
                },
                "phone": {
                    "type": "string"
                }
            },
            "required": [
                "customer_id"
            ]
        }
    },
    {
        "name": "track_order",
        "description": "Track order status using order ID, tracking ID, customer ID, or natural language query about the order",
        "inputSchema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "Specific order ID to look up"
                },
                "tracking_id": {
                    "type": "string",
                    "description": "Tracking ID to look up"
                },
                "customer_id": {
                    "type": "string",
                    "description": "Customer ID to filter orders"
                },
                "natural_query": {
                    "type": "string",
                    "description": "Natural language description of the order (e.g., 'my iPhone order from last Friday')"
                }
            },
            "required": []
        }
    }
]
```

print("[WHITE HEAVY CHECK MARK] Tool schema defined")

## Step 5. Create your AgentCore Gateway

Now let's create the AgentCore Gateway to expose the Lambda function as MCP-compatible endpoint.

To validate the callers authorized to invoke our tools we need to configure the Inbound Auth.

Inbound Auth works using OAuth authorization, the standard for MCP servers. With OAuth the client application must authenticate with the OAuth authorizer before using the Gateway. Your client would receive an access token which is used at runtime.

You need to specify an OAuth discovery server and client IDs. The Cloudformation provided with the workshop already provisioned the Cognito UserPool and UserPoolClient and it stored the discovery URL and the Client ID in dedicated SSM parameters.

In [ ]:
gateway_name = "customersupport-gw"

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            get_ssm_parameter("/app/customersupport/agentcore/machine_client_id")
        ],
        "discoveryUrl": get_ssm_parameter("/app/customersupport/agentcore/cognito_discovery_url")
    }
}

try:
    # create new gateway
    print(f"Creating gateway in region {REGION} with name: {gateway_name}")

    create_response = gateway_client.create_gateway(
        name=gateway_name,
        roleArn= get_ssm_parameter("/app/customersupport/agentcore/gateway_iam_role"),
        protocolType="MCP",
        authorizerType="CUSTOM_JWT",
        authorizerConfiguration=auth_config,
        description="Customer Service Assistant AgentCore Gateway",
    )

    gateway_id = create_response["gatewayId"]

    gateway = {
        "id": gateway_id,
        "name": gateway_name,
        "gateway_url": create_response["gatewayUrl"],
        "gateway_arn": create_response["gatewayArn"],
    }
    put_ssm_parameter("/app/customersupport/agentcore/gateway_id", gateway_id)

    print(f"[WHITE HEAVY CHECK MARK] Gateway created successfully with ID: {gateway_id}")

except Exception as e:
    # If gateway exists, collect existing gateway ID from SSM
    existing_gateway_id = get_ssm_parameter("/app/customersupport/agentcore/gateway_id")
    print(f"Found existing gateway with ID: {existing_gateway_id}")
    
    # Get existing gateway details
    gateway_response = gateway_client.get_gateway(gatewayIdentifier=existing_gateway_id)
    gateway = {
        "id": existing_gateway_id,
        "name": gateway_response["name"],
        "gateway_url": gateway_response["gatewayUrl"],
        "gateway_arn": gateway_response["gatewayArn"],
    }
    gateway_id = gateway['id']

## Step 6. Add the Lambda function Target
Now we will use the previously defined function definitions from [prerequisite/lambda/api_spec.json](./prerequisite/lambda/api_spec.json) to create a Lambda target within our Agent Gateway. This will define the tools that your gateway will host.

Gateway allows you to attach multiple targets to a Gateway and you can change the targets / tools attached to a gateway at any point. Each target can have its own credential provider, but Gateway becomes a single MCP URL enabling access to all of the relevant tools for an agent across myriad APIs.

In [ ]:
def load_api_spec(file_path: str) -> list:
    with open(file_path, "r") as f:
        data = json.load(f)
        
    if not isinstance(data, list):
        raise ValueError("Expected a list in the JSON file")
    return data

try:
    api_spec_file = "./prerequisite/lambda/api_spec.json"

    # Validate API spec file exists
    if not os.path.exists(api_spec_file):
        print(f"[CROSS MARK] API specification file not found: {api_spec_file}")
        sys.exit(1)

    api_spec = load_api_spec(api_spec_file)
 
    # Use Cognito for Inbound OAuth to our Gateway
    lambda_target_config = {
        "mcp": {
            "lambda": {
                "lambdaArn": get_ssm_parameter("/app/customersupport/agentcore/lambda_arn"),
                "toolSchema": {"inlinePayload": api_spec},
            }
        }
    }


    # Create gateway target
    credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]

    create_target_response = gateway_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name="LambdaUsingSDK",
        description="Lambda Target using SDK",
        targetConfiguration=lambda_target_config,
        credentialProviderConfigurations=credential_config,
    )

    print(f"[WHITE HEAVY CHECK MARK] Gateway target created: {create_target_response['targetId']}")

except Exception as e:
    print(f"[CROSS MARK] Error creating gateway target: {str(e)}")

## Step 7: Add our new MCP-based tools to our support agent
Here we integrate our authentication token from Cognito into an MCPClient from Strands SDK to create an MCP Server object to integrate with our Strands Agent

In [ ]:
def get_token(client_id: str, client_secret: str, scope_string: str, url: str) -> dict:
    try:
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        data = {
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret,
            "scope": scope_string,

        }
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()
        return response.json()

    except requests.exceptions.RequestException as err:
        return {"error": str(err)}

print("[WHITE HEAVY CHECK MARK] Fuction to get authentication token defined")

## Step 7.1. Set up a secure MCP client object

In [ ]:
gateway_access_token = get_token(
    get_ssm_parameter("/app/customersupport/agentcore/machine_client_id"),
    get_cognito_client_secret(),
    get_ssm_parameter("/app/customersupport/agentcore/cognito_auth_scope"),
    get_ssm_parameter("/app/customersupport/agentcore/cognito_token_url"))

print(f"Gateway Endpoint - MCP URL: {gateway['gateway_url']}")

# Set up MCP client
mcp_client = MCPClient(
    lambda: streamablehttp_client(
        gateway['gateway_url'],
        headers={"Authorization": f"Bearer {gateway_access_token['access_token']}"},
    )
)

print("[WHITE HEAVY CHECK MARK] MCP client Setup!")

## Step 7.2. Access tools in our agent using our MCP client
Now we will create our Strands Agent using the AgentCore Gateway we built along with the resources from previous labs. Our agent now uses a mix of local tools via our Strands Agent and MCP tools via AgentCore Gateway

In [ ]:
from lab_helpers.lab1_strands_agent import get_product_info, get_return_policy, SYSTEM_PROMPT
from lab_helpers.lab2_memory import CustomerSupportMemoryHooks,create_or_get_memory_resource 
import uuid
from bedrock_agentcore.memory import MemoryClient

memory_client = MemoryClient(region_name=REGION)

memory_id = create_or_get_memory_resource()
SESSION_ID = str(uuid.uuid4())
CUSTOMER_ID = "customer_001"
memory_hooks = CustomerSupportMemoryHooks(memory_id, memory_client, CUSTOMER_ID, SESSION_ID)

# Initialize the Bedrock model
model_id = "us.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
    temperature=0.3,  # Balanced between creativity and consistency
    region_name=REGION
)

try:
    mcp_client.start()
except Exception as e:
    print(f"Error initializing agent: {str(e)}")

tools = (
            [
                get_product_info,
                get_return_policy
            ]
            + mcp_client.list_tools_sync()
        )

# Create the Customer Service Assistant
agent = Agent(
    model=model,
    tools=tools,
    hooks=[memory_hooks],
    system_prompt=SYSTEM_PROMPT
)

print("[WHITE HEAVY CHECK MARK] Customer Service Assistant created successfully!")

## Step 8: Test the agent with MCP tool access to existing APIs”

Let's test our agent with sample queries to ensure all features work correctly.

In [ ]:
test_prompts = [
    # Customer Service Assistant Tests
    "List all of your tools",
    "What's the return policy for smartphones?",
    "Can you tell me about the iPhone 15 Pro specifications?",
    "I want to track my order ORD-2025-001234",
    "What's the status of my iPhone order from last Friday?",
    "Can I return my laptop if I bought it 2 weeks ago?",
    "Tell me about the features of your wireless headphones"
]

# Function to test the agent
def test_agent_responses(agent, prompts):
    for i, prompt in enumerate(prompts, 1):
        print(f"\nTest Case {i}: {prompt}")
        print("-" * 50)
        try:
            response = agent(prompt)
        except Exception as e:
            print(f"Error: {str(e)}")
        print("-" * 50)

# Run the tests
test_agent_responses(agent, test_prompts)

print("\\n[WHITE HEAVY CHECK MARK] Basic testing completed!")


### Congratulations! [PARTY POPPER]


You have successfully completed Lab 3: Securely connect tools to your Agent with AgentCore Gateway

What You Accomplished:

##### Tool Centralization & Reusability:

- Migrated order tracking from local tool to centralized AgentCore Gateway
- Integrated existing enterprise Lambda functions (order tracking, customer profile)
- Created a shared tool infrastructure that multiple agent types can access

##### Enterprise-Grade Security:

- Implemented JWT-based authentication with Cognito integration
- Configured secure inbound authorization for gateway access
- Established identity-based access control for tool usage

##### Scalable Architecture Foundation:

- Built reusable tools that serve multiple use cases (customer service, sales, returns processing)
- Eliminated code duplication across different agents
- Created centralized management for tool updates and maintenance

##### Current Limitations (We'll fix these next!):

- **Local Development Environment** - Still running on your laptop, not production-ready
- **Limited Observability** - No comprehensive monitoring of agent behavior and performance
- **Manual Scaling** - Cannot automatically handle increased load or multiple concurrent users

##### Next Up: [Lab 4 - Deploying to Production with AgentCore Runtime  →](lab-04-agentcore-runtime.ipynb)

In Lab 4, you'll transform your prototype into a production-ready system with:

- AgentCore Runtime for scalable agent deployment
- Comprehensive observability with metrics, logging, and tracing
- Auto-scaling capabilities to handle real-world traffic

### Resources
- [Amazon Bedrock Agent Core Gateway](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway.html)
- [Strands Agents Documentation](https://github.com/strands-agents/sdk-python)
- [Official Customer Service Assistant Sample](https://github.com/awslabs/amazon-bedrock-agentcore-samples/tree/main/02-use-cases/customer-support-assistant)